In [1]:
import numpy as np

In [2]:
import heapq as hq

In [3]:
c1 = "c1"
c2 = "c2"
c3 = "c3"
contents = [c1, c2, c3]

In [4]:
lambda_ = 1

In [5]:
n_events = 1000

In [6]:
# [ref] heapq: https://docs.python.org/3/library/heapq.html#basic-examples
class Cache:
    def __init__(self, id, events, size=2):
        ''' Inicializa cache
        
        Parametros:
            id (str): identificador da cache
            events (tuple): valores inicias a serem contidos na cache
                do tipo (arrival_time, content)
            size (int): capacidade da cache
        '''
        self.id = id
        self._events = events
        hq.heapify(self._events)
        self._size = size
        self._last_arrival = 0
        self.misses = {}

    def next_execution(self):
        return self._events[0]

    def events(self):
        for event in self._events:
            yield event[1]

    def update_entry(self, event):
        ''' Usado apenas no caso da LRU '''
        for i in range(len(self._events)):
            e = self._events[i]
            if event[1] == e[1]:
                self._events[i] = event
        hq.heapify(self._events)

    def add(self, event, cid):
        ''' Adiciona evento (arrival_time, content) a cache'''
        if len(self._events) == self._size:
            if event[1] not in self.events():
                self.misses[cid] = 1
                removed_element = hq.heapreplace(self._events, event)
                # print(f"Elemento {removed_element} removido da cache {self.id}")
                # print(f"Elemento {event} adicionado a cache {self.id}")
            else:
                self.misses[cid] = 0
                # print(f"Element {event} já presente na cache {self.id}")
                # self.update_entry(event)

        else:
            self.misses[cid] = 1
            hq.heappush(self._events, event)
            print(f"Elemento {event} adicionado a cache {self.id}")

        self._last_arrival = max(self._last_arrival, event[0])

In [7]:
# faço broadcast desse conteudo para todas as caches
# mantenho também uma heap de prioridade dos eventos
# TODO: transformar (arrival_time, (cache, content)) em um objeto
def broadcast_content(events, caches, time, content, cid):
    for cache in caches:
        arrival_time = time + np.random.exponential(1/lambda_)
        hq.heappush(events, (arrival_time, (cache, cid, content)))

In [8]:
def simulate_cenario1():
    events = []
    hq.heapify(events)
    
    cache1 = Cache(1, [(0, c1), (0, c2)])
    cache2 = Cache(2, [(0, c1), (0, c2)])
    caches = [cache1, cache2]
    
    # sorteia primeiro conteúdo a ser processado
    content = np.random.choice(contents)
    
    content_id = 0
    broadcast_content(events, caches, 0, content, content_id)
    for _ in range(n_events):
        # processa próximo evento
        time, (cache, cid, content) = hq.heappop(events)
        # print(f"[{time}] Adicionando {content} a cache {cache.id}")
        cache.add((time, content), cid)
    
        # sorteia processo conteudo a ser processado
        content = np.random.choice(contents)
        content_id += 1
        broadcast_content(events, caches, time, content, content_id)
    
    # processa as requisições restantes
    while len(events) > 0:
        # processa próximo evento
        time, (cache, cid, content) = hq.heappop(events)
        # print(f"[{time}] Adicionando {content} a cache {cache.id}")
        cache.add((time, content), cid)
    
    misses = 0
    for cid in range(content_id):
        if sum([cache.misses[cid] for cache in caches]) == len(caches):
            misses += 1
    return misses

In [11]:
n_sims = 100
misses = 0
for _ in range(n_sims):
    misses += simulate_cenario1()

(misses/n_sims)/n_events

0.11255